# 예제 1: 연속 경로 TOYO 데이터 분석

## 📁 분석 대상

ATL Q7M Inner 2C 상온수명 테스트 (1-400 사이클)

**4개 연속 폴더:**
1. `250207_250307_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 1-100cyc`
2. `250219_250319_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 101-200cyc`
3. `250304_250404_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 201-300cyc`
4. `250317_251231_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 301-400cyc`

## 🎯 분석 목표

- 4개 폴더의 사이클 데이터를 연속으로 통합
- 1-400 사이클 전체 수명 분석
- 용량 열화 추세 파악

In [ ]:
# 환경 설정
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# 한글 폰트
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# BatteryDataTool 모듈 Import
sys.path.append(r'c:\Users\Ryu\Python_project\data\battery251027')
from BatteryDataTool import (
    check_cycler, name_capacity,
    toyo_cycle_data
)

print("✅ 환경 설정 완료")

In [ ]:
# 데이터 경로 설정
BASE_PATH = r"C:\Users\Ryu\Python_project\data\BatteryData_251010\Rawdata"

# 연속 경로 리스트 (사이클 순서대로)
folder_paths = [
    os.path.join(BASE_PATH, "250207_250307_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 1-100cyc"),
    os.path.join(BASE_PATH, "250219_250319_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 101-200cyc"),
    os.path.join(BASE_PATH, "250304_250404_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 201-300cyc"),
    os.path.join(BASE_PATH, "250317_251231_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 301-400cyc")
]

# 경로 확인
for i, path in enumerate(folder_paths, 1):
    exists = "✅" if os.path.exists(path) else "❌"
    print(f"{exists} 폴더 {i}: {os.path.basename(path)}")
    if os.path.exists(path):
        cycler_type = "TOYO" if check_cycler(path) == False else "PNE" if check_cycler(path) == True else "UNKNOWN"
        capacity = name_capacity(os.path.basename(path))
        print(f"      충방전기: {cycler_type}, 용량: {capacity} mAh")

In [ ]:
# 각 폴더에서 사이클 데이터 로딩
print("\n=== 사이클 데이터 로딩 중 ===\n")

capacity = 1689  # mAh
ini_crate = 2.0  # 2C rate
chkir = True

cycle_data_list = []

for i, folder_path in enumerate(folder_paths, 1):
    if not os.path.exists(folder_path):
        print(f"⚠️ 폴더 {i}: 경로 없음, 건너뜀")
        continue
    
    try:
        print(f"📂 폴더 {i}: {os.path.basename(folder_path)}")
        
        # TOYO 데이터 로딩
        cycle_df = toyo_cycle_data(folder_path, capacity, ini_crate, chkir)
        
        if cycle_df is not None and len(cycle_df) > 0:
            cycle_data_list.append(cycle_df)
            print(f"   ✅ 로딩 완료: {len(cycle_df)} 사이클")
            print(f"   사이클 범위: {cycle_df.index.min()} ~ {cycle_df.index.max()}")
        else:
            print(f"   ⚠️ 데이터 없음")
    
    except Exception as e:
        print(f"   ❌ 오류: {e}")

print(f"\n✅ 총 {len(cycle_data_list)}개 폴더 로딩 완료")

In [ ]:
# 데이터 통합
if len(cycle_data_list) > 0:
    # 모든 DataFrame 연결
    combined_df = pd.concat(cycle_data_list, axis=0)
    
    # 중복 인덱스 제거 (같은 사이클 번호가 있을 경우)
    combined_df = combined_df[~combined_df.index.duplicated(keep='first')]
    
    # 인덱스 정렬
    combined_df = combined_df.sort_index()
    
    print(f"\n=== 통합 데이터 정보 ===\n")
    print(f"총 사이클 수: {len(combined_df)}")
    print(f"사이클 범위: {combined_df.index.min()} ~ {combined_df.index.max()}")
    print(f"\n데이터 컬럼: {list(combined_df.columns)}")
    print(f"\n데이터 미리보기:")
    print(combined_df.head(10))
else:
    print("⚠️ 로딩된 데이터가 없습니다.")

In [ ]:
# 사이클 분석 그래프
if len(cycle_data_list) > 0:
    fig, axes = plt.subplots(3, 2, figsize=(15, 12))
    fig.suptitle(f'ATL Q7M Inner 2C 상온수명 분석 (1-400 사이클, {capacity} mAh)', 
                 fontsize=16, fontweight='bold')
    
    cycles = combined_df.index.values
    
    # 1. 방전 용량
    axes[0, 0].plot(cycles, combined_df['Dchg'], 'o-', color='#2E86AB', linewidth=2, markersize=3)
    axes[0, 0].set_xlabel('Cycle', fontsize=12)
    axes[0, 0].set_ylabel('방전 용량 비율', fontsize=12)
    axes[0, 0].set_title('1. 방전 용량 변화', fontsize=12, fontweight='bold')
    axes[0, 0].set_ylim(0.70, 1.05)
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].axhline(y=0.8, color='r', linestyle='--', linewidth=1, alpha=0.5, label='80% 기준')
    axes[0, 0].legend()
    
    # 2. 충전 용량
    axes[0, 1].plot(cycles, combined_df['Chg'], 'o-', color='#A23B72', linewidth=2, markersize=3)
    axes[0, 1].set_xlabel('Cycle', fontsize=12)
    axes[0, 1].set_ylabel('충전 용량 비율', fontsize=12)
    axes[0, 1].set_title('2. 충전 용량 변화', fontsize=12, fontweight='bold')
    axes[0, 1].set_ylim(0.70, 1.05)
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. 충방 효율
    axes[1, 0].plot(cycles, combined_df['Eff'], 'o-', color='#F18F01', linewidth=2, markersize=3)
    axes[1, 0].set_xlabel('Cycle', fontsize=12)
    axes[1, 0].set_ylabel('효율', fontsize=12)
    axes[1, 0].set_title('3. 충방 효율 (Dchg/Chg)', fontsize=12, fontweight='bold')
    axes[1, 0].set_ylim(0.95, 1.02)
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. 온도
    axes[1, 1].plot(cycles, combined_df['Temp'], 'o-', color='#BC4B51', linewidth=2, markersize=3)
    axes[1, 1].set_xlabel('Cycle', fontsize=12)
    axes[1, 1].set_ylabel('온도 (°C)', fontsize=12)
    axes[1, 1].set_title('4. 온도 변화', fontsize=12, fontweight='bold')
    axes[1, 1].grid(True, alpha=0.3)
    
    # 5. DCIR
    if 'dcir' in combined_df.columns:
        axes[2, 0].plot(cycles, combined_df['dcir'], 'o-', color='#6A994E', linewidth=2, markersize=3)
        axes[2, 0].set_xlabel('Cycle', fontsize=12)
        axes[2, 0].set_ylabel('DCIR (mΩ)', fontsize=12)
        axes[2, 0].set_title('5. DCIR 변화', fontsize=12, fontweight='bold')
        axes[2, 0].grid(True, alpha=0.3)
    
    # 6. 구간별 비교 (100 사이클 단위)
    section_labels = ['1-100', '101-200', '201-300', '301-400']
    section_avg_cap = []
    
    for i in range(4):
        start_cycle = i * 100 + 1
        end_cycle = (i + 1) * 100
        section_data = combined_df[(combined_df.index >= start_cycle) & (combined_df.index <= end_cycle)]
        if len(section_data) > 0:
            section_avg_cap.append(section_data['Dchg'].mean())
        else:
            section_avg_cap.append(0)
    
    axes[2, 1].bar(section_labels, section_avg_cap, color=['#2E86AB', '#A23B72', '#F18F01', '#C73E1D'])
    axes[2, 1].set_xlabel('사이클 구간', fontsize=12)
    axes[2, 1].set_ylabel('평균 방전 용량 비율', fontsize=12)
    axes[2, 1].set_title('6. 구간별 평균 용량', fontsize=12, fontweight='bold')
    axes[2, 1].set_ylim(0.70, 1.05)
    axes[2, 1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# 통계 분석
if len(cycle_data_list) > 0:
    print("\n" + "="*80)
    print("상세 통계 분석")
    print("="*80 + "\n")
    
    print(f"📊 전체 데이터")
    print(f"   총 사이클: {len(combined_df)}")
    print(f"   사이클 범위: {combined_df.index.min()} ~ {combined_df.index.max()}")
    
    print(f"\n💾 방전 용량")
    initial_cap = combined_df['Dchg'].iloc[0]
    final_cap = combined_df['Dchg'].iloc[-1]
    print(f"   초기: {initial_cap:.4f} ({initial_cap * capacity:.1f} mAh)")
    print(f"   최종: {final_cap:.4f} ({final_cap * capacity:.1f} mAh)")
    print(f"   용량 유지율: {(final_cap / initial_cap) * 100:.2f}%")
    print(f"   용량 손실: {((initial_cap - final_cap) / initial_cap) * 100:.2f}%")
    
    # 80% 도달 사이클 찾기
    below_80 = combined_df[combined_df['Dchg'] < 0.8]
    if len(below_80) > 0:
        eol_cycle = below_80.index[0]
        print(f"   80% EOL 도달: Cycle {eol_cycle}")
    else:
        print(f"   80% EOL: 미도달")
    
    print(f"\n⚡ 효율")
    print(f"   평균 충방 효율: {combined_df['Eff'].mean():.4f} ({combined_df['Eff'].mean() * 100:.2f}%)")
    print(f"   최소 효율: {combined_df['Eff'].min():.4f}")
    print(f"   최대 효율: {combined_df['Eff'].max():.4f}")
    
    print(f"\n🌡️ 온도")
    print(f"   평균 온도: {combined_df['Temp'].mean():.2f} °C")
    print(f"   최저 온도: {combined_df['Temp'].min():.2f} °C")
    print(f"   최고 온도: {combined_df['Temp'].max():.2f} °C")
    print(f"   온도 편차: {combined_df['Temp'].std():.2f} °C")
    
    if 'dcir' in combined_df.columns:
        print(f"\n🔋 DCIR")
        print(f"   평균 DCIR: {combined_df['dcir'].mean():.2f} mΩ")
        print(f"   초기 DCIR: {combined_df['dcir'].iloc[0]:.2f} mΩ")
        print(f"   최종 DCIR: {combined_df['dcir'].iloc[-1]:.2f} mΩ")
        print(f"   DCIR 증가율: {((combined_df['dcir'].iloc[-1] / combined_df['dcir'].iloc[0]) - 1) * 100:.2f}%")
    
    print(f"\n📈 구간별 분석 (100 사이클 단위)")
    for i in range(4):
        start_cycle = i * 100 + 1
        end_cycle = (i + 1) * 100
        section_data = combined_df[(combined_df.index >= start_cycle) & (combined_df.index <= end_cycle)]
        
        if len(section_data) > 0:
            print(f"\n   구간 {i+1} (Cycle {start_cycle}-{end_cycle}):")
            print(f"      사이클 수: {len(section_data)}")
            print(f"      평균 방전 용량: {section_data['Dchg'].mean():.4f}")
            print(f"      평균 효율: {section_data['Eff'].mean():.4f}")
            print(f"      평균 온도: {section_data['Temp'].mean():.2f} °C")
            if 'dcir' in section_data.columns:
                print(f"      평균 DCIR: {section_data['dcir'].mean():.2f} mΩ")

In [ ]:
# Excel 저장 (옵션)
save_excel = True

if save_excel and len(cycle_data_list) > 0:
    output_path = os.path.join(BASE_PATH, "ATL_Q7M_Inner_2C_상온수명_1-400cyc_통합분석.xlsx")
    
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        # 통합 데이터
        combined_df.to_excel(writer, sheet_name='통합_사이클_데이터')
        
        # 통계
        stats_data = {
            '항목': ['총 사이클 수', '초기 용량 비율', '최종 용량 비율', '용량 유지율 (%)', 
                   '평균 효율', '평균 온도 (°C)'],
            '값': [
                len(combined_df),
                f"{combined_df['Dchg'].iloc[0]:.4f}",
                f"{combined_df['Dchg'].iloc[-1]:.4f}",
                f"{(combined_df['Dchg'].iloc[-1] / combined_df['Dchg'].iloc[0]) * 100:.2f}",
                f"{combined_df['Eff'].mean():.4f}",
                f"{combined_df['Temp'].mean():.2f}"
            ]
        }
        stats_df = pd.DataFrame(stats_data)
        stats_df.to_excel(writer, sheet_name='통계', index=False)
    
    print(f"\n✅ Excel 저장 완료: {output_path}")

---
## 📝 분석 결론

### 주요 발견사항

1. **수명 특성**
   - 4개 구간(1-400 사이클) 데이터 통합 분석
   - 용량 열화 추세 파악
   - 80% EOL 도달 시점 확인

2. **구간별 비교**
   - 각 100 사이클 구간의 평균 성능
   - 시간에 따른 열화 패턴

3. **권장사항**
   - 연속 데이터이므로 전체 추세 분석 가능
   - 구간별 변화율 모니터링 필요

---

**작성일**: 2025-10-28  
**데이터**: TOYO 연속 경로 (1-400 사이클)  
**배터리**: ATL Q7M Inner (1689 mAh, 2C, 23°C)